### 导包

In [166]:
import requests
import fake_useragent
from lxml import etree
import re
import time
import pandas as pd
import random
# 首先实例化fake_useragent对象
ua = fake_useragent.UserAgent()

### 测试是否可以getHtml

In [159]:
headers = {
  'User-Agent' : ua.random,
  'Cookie' : 'bid=4w1lEeb_O0I; douban-fav-remind=1; ll="118281"; __utmc=30149280; __utmz=30149280.1704272435.5.4.utmcsr=bing|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); ap_v=0,6.0; __utma=30149280.2051814712.1693206043.1704278261.1704282134.8; __utmt=1; viewed="1421488_3039969_1406094_2995812_3184419_1403307_36660948_25817684_26967826_27028228"; dbcl2="177340493:/1rnLRV8PEc"; ck=K3gX; frodotk_db="00115e72af621c1600341b580e265442"; push_noty_num=0; push_doumail_num=0; __utmv=30149280.17734; __utmb=30149280.20.10.1704282134'
}

In [24]:
res = requests.get(url = r"https://music.douban.com/top250?start=0",headers= headers)

In [25]:
print(res.text)

<!DOCTYPE html>
<html lang="zh-CN" class="ua-windows ua-webkit">
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta name="renderer" content="webkit">
    <meta name="referrer" content="always">
    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />
    <title>
豆瓣音乐 Top 250
</title>
    
    <meta http-equiv="Pragma" content="no-cache">
    <meta http-equiv="Expires" content="Sun, 6 Mar 2005 01:00:00 GMT">
    
    <script >var _head_start = new Date();</script>
    <script src="https://img1.doubanio.com/f/music/b66ed708717bf0b4a005a4d0113af8843ef3b8ff/js/music/lib/jquery-1.11.0.min.js"></script>
    <script src="https://img1.doubanio.com/f/music/3451f4db67b2c8e2f4d8112eee591db9fd236975/js/music/lib/jquery-migrate-1.2.1.js"></script>
    <script src="https://img1.doubanio.com/f/shire/4888ee2fda6812f70a064a51c93b84fde8e4a3c2/js/douban.js"></script>
    <script src="https://img1.doubanio.com/f/shire/2c0c1c6b

get 专辑详细信息的url

In [171]:
def get_url_music(url):
  timeout = random.uniform(5, 15)
  html = requests.get(url, headers=headers, timeout=timeout)
  selector = etree.HTML(html.text)
  music_hrefs = selector.xpath('//a[@class="nbg"]/@href')
  print(music_hrefs)
  for music_href in music_hrefs:
    print(music_href)
    get_music_info(music_href)

get 详细专辑信息

In [145]:
def ifElse(name):
  if len(name) == 0:
    name = '未知'
  else:
    name = name[0].strip()
  return name

def get_music_info(music_href):
  ## 获取页面信息
  html = requests.get(music_href, headers=headers, timeout=30)
  ## 使用xtree解析页面
  selector = etree.HTML(html.text)
  
  name = selector.xpath('//*[@id="wrapper"]/h1/span/text()')[0]

  imgSrc = selector.xpath('//*[@id="mainpic"]/span/a/img/@src')[0]

  author = selector.xpath('//*[@id="info"]/span[2]/span/a/text()')
  if author == []:
    author = selector.xpath('//*[@id="info"]/span[1]/span/a/text()')
    
  book_times = re.findall('发行时间:</span>&nbsp;(.*?)<br', html.text, re.S)
  if len(book_times) == 0:
    book_time = '未知'
  else:
    book_time = book_times[0].strip()

  company = re.findall('出版者:</span>&nbsp;(.*?)<br', html.text, re.S)
  if len(company) == 0:
    company = '未知'
  else:
    company = company[0].strip()

  intro = selector.xpath('//*[@id="link-report"]/span[2]/text()')
  if len(intro) == 0:
    intro = '未知'
  else:
    intro = intro[0].strip()

  info = {'name': name, 'author': author, 'imgSrc': imgSrc, 'book_time': book_time, 'company': company, 'intro': intro}
  
  ## 先用pandas存一下数据
  ## 1.字典转成series 
  ## 2.series append
  newRow = pd.Series(info,name='info')
  # df = df.append(new_row_series)
  global tableTemp
  tableTemp = tableTemp.append(newRow)

### 测试第一页

In [162]:
global tableTemp
tableTemp = pd.DataFrame(columns=['name', 'author', 'imgSrc', 'book_time', 'company', 'intro'])


In [173]:
if __name__ == '__main__':
  urls = ['https://music.douban.com/top250?start={}'.format(str(i)) for i in range(0, 225, 25)]
  for url in urls:
    # 构造urls并循环调用函数
    get_url_music(url)
    # 随机睡眠
    random_seconds = random.uniform(2, 4)
    time.sleep(random_seconds)
print("爬取结束！")

['https://music.douban.com/subject/2995812/', 'https://music.douban.com/subject/3040149/', 'https://music.douban.com/subject/1427374/', 'https://music.douban.com/subject/1403307/', 'https://music.douban.com/subject/3259411/', 'https://music.douban.com/subject/5958397/', 'https://music.douban.com/subject/3184419/', 'https://music.douban.com/subject/3041487/', 'https://music.douban.com/subject/1395089/', 'https://music.douban.com/subject/3390002/', 'https://music.douban.com/subject/2131368/', 'https://music.douban.com/subject/1394765/', 'https://music.douban.com/subject/1396380/', 'https://music.douban.com/subject/2134548/', 'https://music.douban.com/subject/3717116/', 'https://music.douban.com/subject/2131595/', 'https://music.douban.com/subject/3801228/', 'https://music.douban.com/subject/1394590/', 'https://music.douban.com/subject/1431596/', 'https://music.douban.com/subject/1406522/', 'https://music.douban.com/subject/1401853/', 'https://music.douban.com/subject/5351500/', 'https://

In [175]:
tableTemp.to_excel(r"C:\Users\10378\Desktop\douban-250-demo.xlsx")

In [176]:
display(tableTemp)

,name,author,imgSrc,book_time,company,intro
info,We Sing. We Dance. We Steal Things.,[Jason Mraz],https://img3.doubanio.com/view/subject/m/publi...,2008-05-13,Atlantic/WEA,沉淀一年，回归简单生活，用全新能量在第三张原创专辑当中活出自我l 部落格女孩Colbie C...
info,Viva La Vida,[Coldplay],https://img9.doubanio.com/view/subject/m/publi...,2008-06-17,Capitol,专辑的名字来自一位墨西哥艺术家Frida Kahlo给自己的一幅画的命名。
info,华丽的冒险,[陈绮贞],https://img9.doubanio.com/view/subject/m/publi...,2005-09-23,艾迴唱片,未知
info,范特西,[周杰伦],https://img3.doubanio.com/view/subject/m/publi...,2001-09-14,BMG,周杰伦的出现，让人们相信台湾创造本土R&B的可能性；周杰伦的走红，彻底地宣布音乐新声代的来临...
info,後。青春期的詩,[五月天],https://img3.doubanio.com/view/subject/m/publi...,2008-10-23,相信音樂,"五个""憨人""闭关半年 熬出睽违两年爆肝专辑‘作品七号’"
...,...,...,...,...,...,...
info,我的歌声里,[曲婉婷],https://img3.doubanio.com/view/subject/m/publi...,2012-07-01,星外星唱片,未知
info,王菲,[王菲],https://img1.doubanio.com/view/subject/m/publi...,1997-09-30,EMI,在百代的首张唱片是《王菲》（1997），由于上年的国语唱片《浮躁》非常另类，十足商业化的新专...
info,信仰在空中飘扬,[汪峰],https://img1.doubanio.com/view/subject/m/publi...,2009-07-25,大国文化,未知
info,[i],[莫文蔚],https://img1.doubanio.com/view/subject/m/publi...,2002-04-29,新力唱片,新力的莫文蔚，新的莫文蔚 精炼的文字质地、爱情／感觉的新感性
